<a href="https://colab.research.google.com/github/l-e13/wnba/blob/main/wnba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/l-e13/wnba

Cloning into 'wnba'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 29.64 KiB | 1.85 MiB/s, done.


In [ ]:
import pandas as pd

draftdf = pd.read_csv('wnbadraft.csv')
draftdf.head()

,overall_pick,year,team,player,former,college,years_played,games,win_shares,win_shares_40,minutes_played,points,total_rebounds,assists
0,1,2022,Atlanta Dream,Rhyne Howard,NaN,Kentucky,1,34.0,2.9,0.110,31.4,16.2,4.5,2.8
1,2,2022,Indiana Fever,NaLyssa Smith,NaN,Baylor,1,32.0,0.0,-0.001,30.7,13.5,7.9,1.4
2,3,2022,Washington Mystics,Shakira Austin,NaN,Ole Miss,1,36.0,3.1,0.160,21.6,8.7,6.5,0.9
3,4,2022,Indiana Fever,Emily Engstler,NaN,Louisville,1,35.0,0.4,0.024,18.2,5.2,5.2,1.5
4,5,2022,New York Liberty,Nyara Sabally,NaN,Oregon,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning

In [ ]:
# check type
draftdf.dtypes

,0
overall_pick,int64
year,int64
team,object
player,object
former,object
college,object
years_played,int64
games,float64
win_shares,float64
win_shares_40,float64


In [ ]:
# check NaNs
draftdf.isnull().sum()

# replace nans in former
draftdf['former'] = draftdf['former'].fillna('US college')

# replace nans in college
draftdf['college'] = draftdf['college'].fillna('International')

# exploring rows with missing players
draftdf[draftdf['player'].isna()]
draftdf = draftdf.dropna(subset=['player']) # dropping 2 rows with missing values

# replace nans with 0
draftdf[['games', 'win_shares', 'win_shares_40', 'points', 'total_rebounds', 'minutes_played', 'assists']] = draftdf[['games', 'win_shares', 'win_shares_40', 'points', 'total_rebounds', 'minutes_played', 'assists']].fillna(0)

draftdf.isnull().sum()



,0
overall_pick,0
year,0
team,0
player,0
former,0
college,0
years_played,0
games,0
win_shares,0
win_shares_40,0


## Visualizations